### 필요 라이브러리 정의

In [1]:
import pandas as pd
import folium as g
from folium.plugins import MarkerCluster
from haversine import haversine
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

pd.options.display.float_format = '{:.5f}'.format

c:\Users\kki96\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\kki96\miniconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\kki96\miniconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\kki96\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


---

### 데이터 로드

In [2]:
train = pd.read_parquet('./jeju_data/train.parquet')
test = pd.read_parquet('./jeju_data/test.parquet')

In [3]:
Train = train.copy()

---

### feature만드는 함수

In [9]:
def make_year(df):
    dt = df['base_date'].astype('str')
    month_data = pd.to_datetime(dt)
    md = month_data.dt.year
    return md


def make_month(df):
    dt = df['base_date'].astype('str')
    month_data = pd.to_datetime(dt)
    md = month_data.dt.month
    return md


def make_day(df):
    dt = df['base_date'].astype('str')
    month_data = pd.to_datetime(dt)
    md = month_data.dt.day
    return md


def turn_road_rate(df):
    df.loc[(df['start_turn_restricted'] == '있음') & (df['road_rating'] == 107), 'turn_road_rate'] = 0
    df.loc[(df['start_turn_restricted'] == '있음') & (df['road_rating'] == 103), 'turn_road_rate'] = 1
    df.loc[(df['start_turn_restricted'] == '없음') & (df['road_rating'] == 107), 'turn_road_rate'] = 2
    df.loc[(df['start_turn_restricted'] == '있음') & (df['road_rating'] == 106), 'turn_road_rate'] = 3
    df.loc[(df['start_turn_restricted'] == '없음') & (df['road_rating'] == 103), 'turn_road_rate'] = 4
    df.loc[(df['start_turn_restricted'] == '없음') & (df['road_rating'] == 106), 'turn_road_rate'] = 5
    return df['turn_road_rate']


def end_turn_road_rate(df):
    df.loc[(df['end_turn_restricted'] == '있음') & (df['road_rating'] == 107), 'end_turn_road_rate'] = 0
    df.loc[(df['end_turn_restricted'] == '있음') & (df['road_rating'] == 103), 'end_turn_road_rate'] = 1
    df.loc[(df['end_turn_restricted'] == '없음') & (df['road_rating'] == 107), 'end_turn_road_rate'] = 2
    df.loc[(df['end_turn_restricted'] == '있음') & (df['road_rating'] == 106), 'end_turn_road_rate'] = 3
    df.loc[(df['end_turn_restricted'] == '없음') & (df['road_rating'] == 103), 'end_turn_road_rate'] = 4
    df.loc[(df['end_turn_restricted'] == '없음') & (df['road_rating'] == 106), 'end_turn_road_rate'] = 5
    return df['end_turn_road_rate']


def make_dist(df):
    dist = []
    for i in tqdm(range(len(df))):
        start_location = (df['start_latitude'][i], df['start_longitude'][i])
        end_location = (df['end_latitude'][i], df['end_longitude'][i])
        
        dist.append(haversine(start_location, end_location))

    return dist


def make_week(df):
    dt = df['base_date'].astype('str')
    data = pd.to_datetime(dt)

    b_list = []
    
    for i in tqdm(range(len(df))):
        b_list.append(data[i].weekday())
    
    return b_list


def week_mapping(df):
    if df['week'] <= 4:
        val = 0
    else:
        val = 1
    return val


# cyclical continuous features - 24-hour time 주기성을 가지는 데이터를 알맞게 변환
def cyclical_feature(df):
    df['sin_time'] = np.sin(2*np.pi*df.base_hour/24)
    df['cos_time'] = np.cos(2*np.pi*df.base_hour/24)
    
    
def over_max_speed(df):
    df.loc[(df['maximum_speed_limit'] == 30), 'over_max_speed'] = 1
    df.loc[(df['maximum_speed_limit'] == 40), 'over_max_speed'] = 1
    df.loc[(df['maximum_speed_limit'] == 50), 'over_max_speed'] = 0
    df.loc[(df['maximum_speed_limit'] == 60), 'over_max_speed'] = 0
    df.loc[(df['maximum_speed_limit'] == 70), 'over_max_speed'] = 0
    df.loc[(df['maximum_speed_limit'] == 80), 'over_max_speed'] = 0
    
    return df['over_max_speed']


---

### 함수적용

In [6]:
tra = make_dist(train)
train['year'] = make_year(train)
train['month'] = make_month(train)
train['day'] = make_day(train)
train['turn_road_rate'] = turn_road_rate(train)
train['end_turn_road_rate'] = end_turn_road_rate(train)
train['distance'] = tra
train['week'] = make_week(train)
train['week'] = train.apply(week_mapping, axis=1)
train['over_max_speed'] = over_max_speed(train)
# cyclical_feature(train)

# tes = make_dist(test)
# test['month'] = make_month(test)
# test['day'] = make_day(test)
# test['turn_road_rate'] = turn_road_rate(test)
# test['end_turn_road_rate'] = end_turn_road_rate(test)
# test['distance'] = tes
# test['week'] = make_week(test)
# test['week'] = test.apply(week_mapping, axis=1)
# cyclical_feature(test)

100%|██████████| 4701217/4701217 [00:39<00:00, 119146.87it/s]


---

In [13]:
str_col = ['day_of_week', 'start_turn_restricted', 'end_turn_restricted', 'road_name', 'start_node_name', 'end_node_name']
for i in str_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])

    # for label in np.unique(test[i]):
    #     if label not in le.classes_:
    #         le.classes_ = np.append(le.classes_, label)
    # test[i] = le.transform(test[i])


In [14]:
X = train.drop(
    ['id', 'base_date', 'target'], axis=1)

y = train['target']

# test = test.drop(
#     ['id', 'base_date', 'road_name', 'start_node_name', 'end_node_name'], axis=1
# )

In [15]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1103)

In [114]:
# cat_best_parm = {'learning_rate': 0.21020261303127669, 'bagging_temperature': 75.5373108410501, 'n_estimators': 4891, 'max_depth': 12, 'random_strength': 19, 'colsample_bylevel': 0.45647247075797176, 'l2_leaf_reg': 4.82164247062514e-07, 'min_child_samples': 82, 'max_bin': 448, 'od_type': 'IncToDec'}
# xgb_best_param = {'n_estimators': 2626, 'max_depth': 15, 'min_child_weight': 39, 'gamma': 1, 'colsample_bytree': 0.8320047615067258, 'lambda': 8.576465850923702, 'alpha': 1.2726833950057483, 'subsample': 0.8}

In [10]:
# from catboost import CatBoostRegressor
# LR = CatBoostRegressor(iterations=1000).fit(x_train, y_train)
# y_pred = LR.predict(x_val)
# mae = mean_absolute_error(y_val, y_pred)
# print(mae)

In [16]:
LR = XGBRegressor().fit(x_train, y_train)
y_pred = LR.predict(x_val)
mae = mean_absolute_error(y_val, y_pred)
print(mae)

c:\Users\kki96\miniconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


3.807563939209401


In [ ]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("./submit_xgb.csv", index=False)

In [ ]:
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler
# from catboost import CatBoostRegressor

# def objective_xgb(trial: Trial, x_value, y_value):
#     param = {
#         "random_state":42,
#         'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#         "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
#         "max_depth":trial.suggest_int("max_depth", 4, 16),
#         'random_strength' :trial.suggest_int('random_strength', 0, 100),
#         "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
#         "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#         "max_bin": trial.suggest_int("max_bin", 200, 500),
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#     }

#     x_train, x_val, y_train, y_val = train_test_split(x_value, y_value, test_size=0.3, random_state=1103)
#     cat_model = CatBoostRegressor(**param)
#     cat_model = cat_model.fit(x_train, y_train, verbose=False, eval_set=[(x_val, y_val)])
#     score = mean_absolute_error(cat_model.predict(x_val), y_val)
#     return score

# study = optuna.create_study(direction='minimize', sampler=TPESampler())
# study.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=30)
# print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))